In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import h5py
import pickle

sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from analysis_utils import envs

%matplotlib inline
plt.style.use('/mnt/home/tnguyen/default.mplstyle')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
def get_mw_idx(cat, h=0.6909):
    """ Function to get MW index from the FoF catalog """

    masses = cat['GroupMassType'][:] * 1e10 / h
    tot_masses = np.sum(cat['GroupMassType'][:],axis=1) * 1e10 / h
    mcut = (tot_masses > 7e11) & (tot_masses < 3e12)
    if np.sum(mcut) == 0:
        return None
    cont = masses[:,2] / tot_masses
    idx = np.argmin(cont[mcut])
    if cont[mcut][idx] > .1:
        return None
    mw_idx = np.arange(len(masses))[mcut][idx]

    return mw_idx

def get_all_params():

    sample = np.loadtxt("/mnt/ceph/users/jrose/MW_zooms/fixed_cosmo/sobol_params.txt")

    Min = np.array([1/30, 0.25, 0.5, 0.25])
    Max = np.array([1/1.8, 4.0, 2.0, 4.0])

    Min[1:] = np.log10(Min[1:])
    Max[1:] = np.log10(Max[1:])

    params = Min + sample*(Max-Min)
    params[:,1:] = np.power(10, params[:,1:])
    params *= np.array([1, 3.6, 7.4, .1])
    params[:,0] = 1/params[:,0]

    return params

In [27]:
root_dir = envs.DEFAULT_FOF_CATALOG
save_dir = envs.DEFAULT_RAW_DATASET_DIR
save_name = "mw_dmo_zooms-wdm.pkl"
use_dmo = True
run_type = 'dmo_zoom' if use_dmo else 'zoom'
snap_num = 0 if use_dmo else 90
scale_factor = 1.0
num_boxes = 1100
sobol_params = get_all_params()
halo_parameters = ['GroupPos', 'GroupVel', 'GroupMass', 'GroupMassType']
subhalo_parameters = [
    'SubhaloPos', 'SubhaloVel', 'SubhaloMass', 'SubhaloMassType', 
    'SubhaloMassInHalfRad', 'SubhaloMassInHalfRadType', 'SubhaloHalfmassRad', 
    'SubhaloHalfmassRadType', 'SubhaloVmaxRad', 'SubhaloVmax',
]
os.makedirs(save_dir, exist_ok=True)

In [28]:
catalog = {p: [] for p in halo_parameters + subhalo_parameters}
catalog['sobol_params'] = []
catalog['box_num'] = []

for box_num in range(num_boxes):
    fof_name = "box_{:d}/{}/RUNs/output/fof_subhalo_tab_{:03d}.hdf5".format(
        box_num, run_type, snap_num)
    fof_path = os.path.join(root_dir, fof_name)

    # print("Reading box {:d}...".format(box_num))
    if not os.path.exists(fof_path):
        print(f"Box {box_num} does not exist. Skipping...")
        continue

    with h5py.File(fof_path, 'r') as f:
        mw_idx = get_mw_idx(f['Group'], f['Parameters'].attrs['HubbleParam'])
        if mw_idx is None:
            print("MW not found in box {:d}...".format(box_num))
            continue
        
        # add halo parameters
        for p in halo_parameters:
            catalog[p].append(f['Group'][p][mw_idx])

        # add subhalo parameters
        select = (f['Subhalo/SubhaloGrNr'][:] == mw_idx)
        for p in subhalo_parameters:
            catalog[p].append(f['Subhalo'][p][select])

        # add sobol parameters and box number
        catalog['sobol_params'].append(sobol_params[box_num])
        catalog['box_num'].append(box_num)

# convert to numpy arrays
for p in catalog:
    catalog[p] = np.array(catalog[p])

# save the catalog to a file
with open(os.path.join(save_dir, save_name), 'wb') as f:
    pickle.dump(catalog, f)

Box 0 does not exist. Skipping...
MW not found in box 42...
MW not found in box 317...
Box 501 does not exist. Skipping...
Box 509 does not exist. Skipping...
Box 519 does not exist. Skipping...
Box 520 does not exist. Skipping...
Box 521 does not exist. Skipping...
MW not found in box 529...
Box 537 does not exist. Skipping...
Box 545 does not exist. Skipping...
Box 549 does not exist. Skipping...
Box 573 does not exist. Skipping...
Box 578 does not exist. Skipping...
Box 601 does not exist. Skipping...
Box 602 does not exist. Skipping...
Box 603 does not exist. Skipping...
Box 604 does not exist. Skipping...
Box 605 does not exist. Skipping...
Box 606 does not exist. Skipping...
Box 607 does not exist. Skipping...
Box 608 does not exist. Skipping...
Box 609 does not exist. Skipping...
Box 610 does not exist. Skipping...
Box 611 does not exist. Skipping...
Box 612 does not exist. Skipping...
Box 613 does not exist. Skipping...
Box 614 does not exist. Skipping...
Box 615 does not exist

/tmp/ipykernel_3573906/1537851740.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  catalog[p] = np.array(catalog[p])
